In [ ]:
# # 读取数据并分割数据
# import pandas as pd
# import os·
# current_dir = os.getcwd()
# data_dir = os.path.join(current_dir, 'data/weibo_senti_100k.csv')
# df = pd.read_csv(data_dir)
# import numpy as np
# num_rows = len(df)
# random_data = np.random.choice(np.arange(num_rows), num_rows, replace=False)
# split_size = int(num_rows * 0.8)
# df1 = df.iloc[random_data[:split_size],:]
# df2 = df.iloc[random_data[split_size:],:]
# df1.to_csv(os.path.join(current_dir,'./data/test.csv'),index=False)
# df2.to_csv(os.path.join(current_dir,'./data/train.csv'),index=False)

In [1]:
# 读取test数据
import os
import pandas as pd
current_dir = os.getcwd()
test_dir = os.path.join(current_dir, 'data/test.csv')
df = pd.read_csv(test_dir)
df.sample(5)


,label,review
9187,0,成都在召唤我[泪]
34075,1,大厨很给力~！哈哈哈?~ 真没什么不能播的，但是千呼万唤始出来 真在说明压轴啊有木有~！...
59161,1,走漏眼，未去?！//@KC?味??：回复 @爱红酒的闷蛋:或者先吃吃（透支定先）再游也可以呀...
4353,1,我能说我是僵尸迷么 [哈哈] #360首发植物大战僵尸2#
65121,1,奶奶的，你终于给老子下来了！一身轻松！[嘻嘻][嘻嘻][哈哈][哈哈][太阳][太阳]


In [3]:
# 测试先用小部分数据
lens = int(len(df))
sentences = df.review[0:lens]
labels = df.label[0:lens]
len(sentences)


95990

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
print('Original:',sentences[1])
print('Tokenized:',tokenizer.tokenize(sentences[1]))
print('Token IDs:',tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[1])))

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001D75B1CAA00>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: ee245a59-8f91-4696-a0be-75e3dc792662)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/tokenizer_config.json


Original: 回复@黛斯CHAN与小胖胖:亲…给点想象力好么？[汗]#与其相濡以沫…不如相望于江湖#[哈哈][哈哈][哈哈] //@黛斯CHAN与小胖胖:第五个菜名叫 尸首异处？？[晕][晕] //@懒人业余餐厅:我比较关心老板疯了没？@懒人业余餐厅厦门店
Tokenized: ['回', '复', '@', '黛', '斯', '[UNK]', '与', '小', '胖', '胖', ':', '亲', '[UNK]', '给', '点', '想', '象', '力', '好', '么', '？', '[', '汗', ']', '#', '与', '其', '相', '濡', '以', '沫', '[UNK]', '不', '如', '相', '望', '于', '江', '湖', '#', '[', '哈', '哈', ']', '[', '哈', '哈', ']', '[', '哈', '哈', ']', '/', '/', '@', '黛', '斯', '[UNK]', '与', '小', '胖', '胖', ':', '第', '五', '个', '菜', '名', '叫', '尸', '首', '异', '处', '？', '？', '[', '晕', ']', '[', '晕', ']', '/', '/', '@', '懒', '人', '业', '余', '餐', '厅', ':', '我', '比', '较', '关', '心', '老', '板', '疯', '了', '没', '？', '@', '懒', '人', '业', '余', '餐', '厅', '厦', '门', '店']
Token IDs: [1726, 1908, 137, 7950, 3172, 100, 680, 2207, 5523, 5523, 131, 779, 100, 5314, 4157, 2682, 6496, 1213, 1962, 720, 8043, 138, 3731, 140, 108, 680, 1071, 4685, 4091, 809, 3773, 100, 679, 1963, 4685, 3307, 754, 3736, 3959, 108, 138, 1506, 1506, 140, 138, 1506, 1506, 140, 

In [5]:
# 查看句子最大长度
max_len = 0
for sentence in sentences:
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
print('Max sentence length:', max_len)

Max sentence length: 195


In [6]:
import torch
# 创建一个空列表，用于存储输入的句子
input_ids =  []
# 创建一个空列表，用于存储注意力掩码
attention_masks = []
# 遍历句子列表
for sentence in sentences:
    # 使用tokenizer对句子进行编码，并返回编码结果和注意力掩码
    encoded_dict = tokenizer.encode_plus(
        sentence,
        add_special_tokens = True,
        max_length = 64,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt',
    )
    # 将编码结果和注意力掩码添加到列表中
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# 将列表中的元素拼接在一起
input_ids = torch.cat(input_ids,dim = 0)
attention_masks = torch.cat(attention_masks,dim = 0)
# 将标签转换为tensor格式
labels = torch.tensor(labels)
# 打印句子、编码结果、注意力掩码和标签
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])
print('Attention mask:', attention_masks[0])
print('Label:', labels[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
d:\Tools\anaconda\envs\pytorch\lib\site-packages\transformers\tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  下午，小憩一小时，由始至终做一梦，向@李峥嵘 同学汇报工作，达人啊，表格啊，带着微博去旅行啊。。[泪][泪][泪]
Token IDs: tensor([ 101,  678, 1286, 8024, 2207, 2737,  671, 2207, 3198, 8024, 4507, 1993,
        5635, 5303,  976,  671, 3457, 8024, 1403,  137, 3330, 2286, 2318, 1398,
        2110, 3726, 2845, 2339,  868, 8024, 6809,  782, 1557, 8024, 6134, 3419,
        1557, 8024, 2372, 4708, 2544, 1300, 1343, 3180, 6121, 1557,  511,  511,
         138, 3801,  140,  138, 3801,  140,  138, 3801,  140,  102,    0,    0,
           0,    0,    0,    0])
Attention mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])
Label: tensor(0)


In [7]:
# 加载数据集
from torch.utils.data import TensorDataset, random_split
# 将输入id，注意力掩码和标签封装成TensorDataset
dataset = TensorDataset(input_ids,attention_masks,labels)
# 计算训练集和验证集的大小
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
# 将数据集按照训练集和验证集的比例划分
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

67,193 training samples
28,797 validation samples


In [8]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# 这个值最好取16或者32
batch_szie = 16
# 创建训练数据加载器
train_dataloader = DataLoader(
    train_dataset,
    # 使用随机采样器，每次迭代都会返回一个新的随机序列
    sampler = RandomSampler(train_dataset),
    # 设置每个batch的大小
    batch_size = batch_szie
)
# 创建验证数据加载器
validation_dataloader = DataLoader(
    val_dataset,
    # 使用顺序采样器，每次迭代都会返回一个新的序列
    sampler=SequentialSampler(val_dataset),
    # 设置每个batch的大小
    batch_size=batch_szie
)

In [9]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# 加载Bert预训练模型，并设置输出层数和类别数
model = BertForSequenceClassification.from_pretrained(
    'bert-base-chinese',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)
# 将模型移动到GPU上
model.cuda()

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001D75B6F8B80>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: b9e4a105-2179-44da-96df-6db838c42cdc)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/config.json
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001D75B6F88B0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 976d69db-3f6e-475c-a1f6-13ca4196d41f)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/config.json
Some weights of BertForSequenceClassification were not initialized from the 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
# 定义优化器
optimer = AdamW(model.parameters(),
                lr = 5e-5,
                eps = 1e-8)
# 定义学习率调度器
from transformers import get_linear_schedule_with_warmup
epochs = 3
total_steps =  len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimer,
                                            num_warmup_steps = 0,
                                            num_training_steps=total_steps)


d:\Tools\anaconda\envs\pytorch\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
import datetime
# 定义一个函数，用于格式化时间
def format_time(elapsed):
    # 将时间elapsed四舍五入到秒
    elapsed_rounded = int(round((elapsed)))
    # 返回格式化后的时间
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [12]:
import numpy as np

# 根据预测结果和标签数据来计算准确率
def flat_accuracy(preds, labels):
    # 将预测结果转换为一维数组
    pred_flat = np.argmax(preds, axis=1).flatten()
    # 将标签数据转换为一维数组
    labels_flat = labels.flatten()
    # 返回预测结果和标签数据的准确率
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [13]:
import random
import time
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed_val = 42 
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []
total_t0 = time.time()
# 开始训练
for epoch_i in range(0,epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elasped = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elasped))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        loss, logits = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels).loss,model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels).logits
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimer.step()
        scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            loss, logits = model(b_input_ids,
                                 token_type_ids=None,
                                 attention_mask=b_input_mask,
                                 labels=b_labels).loss,model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels).logits

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # 统计本次 epoch 的 loss
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # 统计本次评估的时长
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # 记录本次 epoch 的所有统计信息
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))









======== Epoch 1 / 3 ========
Training...
  Batch    40  of  4,200.    Elapsed: 0:00:09.
  Batch    80  of  4,200.    Elapsed: 0:00:14.
  Batch   120  of  4,200.    Elapsed: 0:00:19.
  Batch   160  of  4,200.    Elapsed: 0:00:24.
  Batch   200  of  4,200.    Elapsed: 0:00:29.
  Batch   240  of  4,200.    Elapsed: 0:00:35.
  Batch   280  of  4,200.    Elapsed: 0:00:40.
  Batch   320  of  4,200.    Elapsed: 0:00:45.
  Batch   360  of  4,200.    Elapsed: 0:00:50.
  Batch   400  of  4,200.    Elapsed: 0:00:55.
  Batch   440  of  4,200.    Elapsed: 0:01:00.
  Batch   480  of  4,200.    Elapsed: 0:01:05.
  Batch   520  of  4,200.    Elapsed: 0:01:11.
  Batch   560  of  4,200.    Elapsed: 0:01:16.
  Batch   600  of  4,200.    Elapsed: 0:01:21.
  Batch   640  of  4,200.    Elapsed: 0:01:26.
  Batch   680  of  4,200.    Elapsed: 0:01:32.
  Batch   720  of  4,200.    Elapsed: 0:01:37.
  Batch   760  of  4,200.    Elapsed: 0:01:42.
  Batch   800  of  4,200.    Elapsed: 0:01:47.
  Batch   840  of

In [14]:
# 保存模型
import os

# 模型存储到的路径
output_dir = os.path.join(current_dir,'weibo_model_save/')

# 目录不存在则创建
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# 使用 `save_pretrained()` 来保存已训练的模型，模型配置和分词器
# 它们后续可以通过 `from_pretrained()` 加载
model_to_save = model.module if hasattr(model, 'module') else model  # 考虑到分布式/并行（distributed/parallel）训练
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to d:\Project\python\deep learning\Bert\weibo_classification\weibo_model_save/


('d:\\Project\\python\\deep learning\\Bert\\weibo_classification\\weibo_model_save/tokenizer_config.json',
 'd:\\Project\\python\\deep learning\\Bert\\weibo_classification\\weibo_model_save/special_tokens_map.json',
 'd:\\Project\\python\\deep learning\\Bert\\weibo_classification\\weibo_model_save/vocab.txt',
 'd:\\Project\\python\\deep learning\\Bert\\weibo_classification\\weibo_model_save/added_tokens.json')

In [15]:
# 加载训练统计到 DataFrame 中
df_stats = pd.DataFrame(data=training_stats)

# 使用 epoch 值作为每行的索引
df_stats = df_stats.set_index('epoch')

# 展示表格数据
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.242259,0.235836,0.922804,0:09:05,0:01:25
2,0.265279,0.258744,0.927388,0:08:56,0:01:23
3,0.202896,0.199578,0.932318,0:08:44,0:01:22


In [1]:
# 预测一段评论
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# 加载预训练的BERT模型和微调后的二分类模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('./weibo_model_save').to(device)
tokenizer = BertTokenizer.from_pretrained('./weibo_model_save')

# 对输入的句子进行分词和编码
text = '''
我！绝对！是！个！奇！葩！好一番折腾！千回百转「无数坎坷辛酸好么！！」终于买到了机票候机了！
话说我穿着棉袄在冰冷的办公室忙活啊！今天龙抬头我准备理发的好么！忍了好多天了好么！所以想5点再飞！
深圳！我披头散发穿着棉袄来了！感谢贵人七妙！同事说我靠奇迹和好运活到今天的！[泪][泪][泪]
'''
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
# 使用BERT模型对输入进行编码
with torch.no_grad():
  outputs = model(input_ids)
# 获取概率分布
logits = outputs[0]
probabilities = torch.softmax(logits, dim=-1).tolist()[0]

# 输出预测结果
print('预测结果概率：',probabilities)
if(probabilities[0]>probabilities[1]):
    print('评论为负向评论，预测结果为：负向评论')
else:
    print('评论为正向评论，预测结果为：正向评论')

预测结果概率： [0.9955440163612366, 0.004456047900021076]
评论为负向评论，预测结果为：负向评论
